In [4]:
%load_ext autoreload
%autoreload 2

import molpy as mp
import numpy as np
from pathlib import Path

data_path = Path("data/case2")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
reacter = mp.reacter.LammpsReacter(
    mp.typifier.AmberToolsTypifier("gaff", "bcc"),
)

struct_hd = mp.Struct.from_frame(
    mp.io.read_pdb(data_path / "hexamethylenediamine.pdb"), "hd"
)
struct_aa = mp.Struct.from_frame(mp.io.read_pdb(data_path / "adipic_acid.pdb"), "aa")

In [ ]:
struct_rx1_stp1 = mp.Struct.concat("rx1_stp1", [struct_hd, struct_aa])

Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
                          bond  : only read in bond type 
                   -m     multiplicity (2S+1), default is 1
                   -rn    residue name, overrides input file, default is MOL
                   -rf    residue toplogy file name in prep input f

In [ ]:
init1 = struct_hd.atoms[0]
init2 = struct_aa.atoms[0]
end1 = struct_hd.atoms[8]
end2 = struct_aa.atoms[2]

struct_stp1 = reacter.react(
    "rx1_stp1",
    struct_rx1_stp1,
    init1, init2,
    end1, end2,
    workdir=data_path,
)
mp.io.write_pdb(data_path / "stp1.data", struct_stp1.to_frame())

In [9]:
import random

typifier = mp.typifier.AmberToolsTypifier("gaff", "bcc")
typifier.get_forcefield(struct_hd, workdir=data_path/"hd")
typifier.get_forcefield(struct_aa, workdir=data_path/"aa")

system = mp.System()
system.forcefield = mp.ForceField.merge([struct_hd.forcefield, struct_aa.forcefield])
for i in range(10):
    for j in range(10):
        system.add_struct(struct_hd.move(np.random.random(3) * 20))
        system.add_struct(struct_aa.move(np.random.random(3) * 20))

mp.io.write_lammps(system, data_path / "system.data")

ac_path: data/case2/hd/hd.ac is exists: True
Usage: antechamber -i     input file name
                   -fi    input file format
                   -o     output file name
                   -fo    output file format
                   -c     charge method
                   -cf    charge file name
                   -nc    net molecular charge (int)
                   -a     additional file name
                   -fa    additional file format
                   -ao    additional file operation
                          crd   : only read in coordinate
                          crg   : only read in charge
                          radius: only read in radius
                          name  : only read in atom name
                          type  : only read in atom type
                          bond  : only read in bond type 
                   -m     multiplicity (2S+1), default is 1
                   -rn    residue name, overrides input file, default is MOL
                   -rf

TypeError: BondType.__init__() got multiple values for argument 'label'